In [ ]:
import os
import matplotlib.pyplot as plt
import flatbuffers
import tensorflow as tf
import pathlib
import numpy as np
import pandas as pd

import os

from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras import Model
from tensorflow.keras import Input
from tensorflow.keras import Sequential
from tensorflow.keras.layers import RandomFlip
from tensorflow.keras.layers import RandomRotation
from tensorflow.keras.layers import RandomZoom
from tensorflow.keras.layers import RandomTranslation
from tensorflow.keras.layers import Rescaling
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Normalization
from tensorflow.keras.callbacks import LearningRateScheduler

from tflite_support import metadata_schema_py_generated as _metadata_fb
from tflite_support import metadata as _metadata
import math

In [ ]:
train_dataset = image_dataset_from_directory(directory="./train", image_size=(224, 224))
val_dataset = image_dataset_from_directory(directory="./val", image_size=(224, 224))
test_dataset = image_dataset_from_directory(directory="./test", image_size=(224, 224))

In [ ]:
class_names = train_dataset.class_names
print(class_names)

In [ ]:
label_file_path = 'labels.txt'
with open(label_file_path, 'w') as f:
    f.write('\n'.join(class_names))

In [ ]:
augmentation_layers = Sequential([
   Input(shape=(224, 224, 3)),
   RandomFlip(mode="horizontal"),
   RandomRotation(factor=0.1),
   RandomZoom(height_factor=(-0.2, 0.2)),
   RandomTranslation(height_factor=0.2, width_factor=0.2)
])

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_dataset.take(1):
    for i in range(9):
        augmented_images = augmentation_layers(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:
num_classes = len(class_names)

In [ ]:
inputs = Input(shape=(224, 224, 3))
x = RandomFlip(mode="horizontal")(inputs)
x = RandomRotation(factor=0.1)(x)
x = RandomZoom(height_factor=(-0.2, 0.2))(x)
x = RandomTranslation(height_factor=0.2, width_factor=0.2)(x)

x = Rescaling(scale=1./255)(x)

x = Normalization()(x)

x = Conv2D(filters=128, kernel_size=(3, 3), activation="relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(filters=64, kernel_size=(3, 3), activation="relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)

x = Dropout(0.2)(x)
x = Dense(256, activation="relu")(x)

outputs = Dense(num_classes, activation="softmax")(x)
model = Model(inputs, outputs)

In [ ]:
model.compile(optimizer=RMSprop(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_dataset, epochs=60,
                validation_data=val_dataset,
                callbacks=[EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)],
                verbose=0)

In [ ]:
pd.DataFrame(history.history).plot()

In [ ]:
path = './horsetest.jpg'

img = tf.keras.utils.load_img(
    path, target_size=(224, 224)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)

print(
    "This image most likely belongs to {} with a {} percent confidence."
    .format(class_names[np.argmax(predictions[0])], 100 * np.max(predictions[0]))
)

In [ ]:
path = './closeofdaytest.jpg'

img = tf.keras.utils.load_img(
    path, target_size=(224, 224)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)
test_acc

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# quantization, set the optimization mode and data type
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float32]

tflite_model = converter.convert()

# Save the model.
model_file = 'artmodel.tflite'
with open(model_file, 'wb') as f:
  f.write(tflite_model)

In [ ]:
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "ART-Guide"
model_meta.description = ("Identify the most prominent object in the "
                          "image from a set of %d categories." %
                          num_classes)
model_meta.version = "v1"
model_meta.author = "Sharmin_Akter"

In [ ]:
# Creates input info.
input_meta = _metadata_fb.TensorMetadataT()
input_meta.name = "image"
input_meta.description = (
    "Input image to be classified. The expected image is {0} x {1}, with "
    "three channels (red, blue, and green) per pixel. Each value in the "
    "tensor is a single byte between 0 and 255.".format(224, 224))
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (
    _metadata_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.ImageProperties)
input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (
    _metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = [0.0]
input_normalization.options.std = [1.0]
input_meta.processUnits = [input_normalization]
input_stats = _metadata_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]
input_meta.stats = input_stats

In [ ]:
output_meta = _metadata_fb.TensorMetadataT()
output_meta.name = "probability"
output_meta.description = "Probabilities of the {} labels respectively.".format(num_classes)
output_meta.content = _metadata_fb.ContentT()
output_meta.content.content_properties = _metadata_fb.FeaturePropertiesT()
output_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_stats = _metadata_fb.StatsT()
output_stats.max = [1.0]
output_stats.min = [0.0]
output_meta.stats = output_stats
label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename(label_file_path)
label_file.description = "Labels for objects that the model can recognize."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_AXIS_LABELS
output_meta.associatedFiles = [label_file]

In [ ]:
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [output_meta]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

In [ ]:
# copies model_file to export_path
export_model_path = "ArtGuideModel_with_metadata.tflite"
tf.io.gfile.copy(model_file, export_model_path, overwrite=True)

# populates metadata
populator = _metadata.MetadataPopulator.with_model_file(export_model_path)
populator.load_metadata_buffer(metadata_buf)
populator.load_associated_files([label_file_path])
populator.populate()

In [ ]:
interpreter = tf.lite.Interpreter(model_path=export_model_path)

In [ ]:
print("n--------Input Details of Model-------------------n")
input_details = interpreter.get_input_details()
print(input_details)

print("\nn--------Output Details of Model-------------------n")
output_details = interpreter.get_output_details()
print(output_details)

In [ ]:
interpreter.allocate_tensors()

In [ ]:
interpreter.set_tensor(input_details[0]['index'], img_array)

In [ ]:
interpreter.invoke()

In [ ]:
predictions = interpreter.get_tensor(output_details[0]['index'])
score_lite = tf.nn.softmax(predictions)
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score_lite)], 100 * np.max(score_lite))
)